In [48]:
from typing import List, Optional
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """조건에 따라 메시지를 저장"""
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            self.messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            self.messages.extend(output_messages)
        elif self.save_mode == "both":
            self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

In [49]:
store = {}

def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]

In [50]:
from typing import Optional, Dict, Any
from langchain_core.runnables import Runnable

class RunnableWithMessageHistory(Runnable):
    def __init__(self, runnable: Runnable, get_session_history, input_messages_key: str, history_messages_key: str, context_key: Optional[str] = None):
        self.runnable = runnable
        self.get_session_history = get_session_history
        self.input_messages_key = input_messages_key
        self.history_messages_key = history_messages_key
        self.context_key = context_key
    
    def invoke(self, input: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Any:
        session_id = config["configurable"]["session_id"]
        history = self.get_session_history(session_id)
        
        current_input = input[self.input_messages_key]
        
        if isinstance(current_input, str):
            current_input_message = HumanMessage(content=current_input)
            history.add_messages([current_input_message])
        
        input[self.history_messages_key] = history.messages
        
        result = self.runnable.invoke(input, config)
        
        if isinstance(result, AIMessage):
            if self.context_key and self.context_key in input:
                context = input[self.context_key]
                result_with_context = AIMessage(content=f"{context}\n{result.content}")
                history.add_messages([result_with_context])
            else:
                history.add_messages([result])
        
        return result

In [51]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into one of the following two types:
            - Product inquiry, order history inquiry, order change history inquiry, order cancellation history inquiry: '문의'
            - Order request, order change request, order cancellation request: '요청'
            
            You need to review the messages in the Messages Placeholder from the latest to the oldest.

            Consider the previous AI responses and their classifications to understand the intent behind the current input. 
            Use this context to make an accurate classification. 
            If the latest AI response was classified as '요청', and the current input is related to an order, it is likely a '요청'.
            
            Additionally, if the input contains order details, it should be classified as '요청'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [52]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [53]:
classify_message_chain = message_type_prompt | model

In [55]:
classify_message_chain_with_history = RunnableWithMessageHistory(
    classify_message_chain,
    get_by_session_id,
    input_messages_key="input",
    history_messages_key="chat_history",
    context_key="context"
)

In [26]:
result = classify_message_chain_with_history.invoke(
    {"input": "주문 변경할게요", "context": "사용자 입력에 대한 분류"},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)

content='요청' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 192, 'total_tokens': 195}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4303dd41-6575-4098-92b1-8ccd0457f585-0'


In [27]:
store["session_1"].messages

[HumanMessage(content='주문 변경할게요'), AIMessage(content='사용자 입력에 대한 분류\n요청')]

In [28]:
result = classify_message_chain_with_history.invoke(
    {"input": "판매 상품 좀 알려줘", "context": "사용자 입력에 대한 분류"},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)

content='사용자 입력에 대한 분류\n문의' response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 240, 'total_tokens': 253}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-42570256-7f8a-440d-a095-944895ef10d5-0'


context로 입력한 내용(사용자 입력에 대한 분류)이 중복되어 두 번 입력됨

In [16]:
store["session_1"].messages

[HumanMessage(content='주문 변경할게요'),
 AIMessage(content='사용자 입력에 대한 분류\n요청'),
 HumanMessage(content='판매 상품 좀 알려줘'),
 AIMessage(content='사용자 입력에 대한 분류\n사용자 입력에 대한 분류\n문의')]

In [17]:
result = classify_message_chain_with_history.invoke(
    {"input": "주문 취소 가능한가요?", "context": "사용자 입력에 대한 분류"},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)

content='사용자 입력에 대한 분류\n문의' response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 276, 'total_tokens': 289}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7c1babe8-b8b2-4912-a84b-1355465ee7b4-0'


In [19]:
store["session_1"].messages

[HumanMessage(content='주문 변경할게요'),
 AIMessage(content='사용자 입력에 대한 분류\n요청'),
 HumanMessage(content='판매 상품 좀 알려줘'),
 AIMessage(content='사용자 입력에 대한 분류\n사용자 입력에 대한 분류\n문의'),
 HumanMessage(content='주문 취소 가능한가요?'),
 AIMessage(content='사용자 입력에 대한 분류\n사용자 입력에 대한 분류\n문의')]

In [20]:
result = classify_message_chain_with_history.invoke(
    {"input": "주문 변경 가능한가요?", "context": ""},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)

content='사용자 입력에 대한 분류\n문의' response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 314, 'total_tokens': 327}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-82532bc9-3221-461c-9c26-caa0bf95006b-0'


In [21]:
store["session_1"].messages

[HumanMessage(content='주문 변경할게요'),
 AIMessage(content='사용자 입력에 대한 분류\n요청'),
 HumanMessage(content='판매 상품 좀 알려줘'),
 AIMessage(content='사용자 입력에 대한 분류\n사용자 입력에 대한 분류\n문의'),
 HumanMessage(content='주문 취소 가능한가요?'),
 AIMessage(content='사용자 입력에 대한 분류\n사용자 입력에 대한 분류\n문의'),
 HumanMessage(content='주문 변경 가능한가요?'),
 AIMessage(content='\n사용자 입력에 대한 분류\n문의')]

In [22]:
result = classify_message_chain_with_history.invoke(
    {"input": "주문 변경 쌉가능??", "context": ""},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)

content='사용자 입력에 대한 분류\n요청' response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 346, 'total_tokens': 360}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1cf5526f-2811-43e6-8c92-b19bcdfe276d-0'


왜 콘텍스트 공백 문자 넣어도 이전과 맥락 추가하는 거지? 이전 대화 보고 패턴 파악해서 자동생성하는 건가?

In [ ]:
store["session_1"].messages

[HumanMessage(content='주문 변경할게요'),
 AIMessage(content='사용자 입력에 대한 분류\n요청'),
 HumanMessage(content='판매 상품 좀 알려줘'),
 AIMessage(content='사용자 입력에 대한 분류\n사용자 입력에 대한 분류\n문의'),
 HumanMessage(content='주문 취소 가능한가요?'),
 AIMessage(content='사용자 입력에 대한 분류\n사용자 입력에 대한 분류\n문의'),
 HumanMessage(content='주문 변경 가능한가요?'),
 AIMessage(content='\n사용자 입력에 대한 분류\n문의'),
 HumanMessage(content='주문 변경 쌉가능??'),
 AIMessage(content='\n사용자 입력에 대한 분류\n요청')]

---
# helper 코드 동작 점검

In [77]:
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import Runnable, RunnableLambda, RunnablePassthrough
from typing import Optional, Dict, Any

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """조건에 따라 메시지를 저장"""
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            self.messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            self.messages.extend(output_messages)
        elif self.save_mode == "both":
            self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

store = {}
def get_session_history(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]

class RunnableWithMessageHistory(Runnable):
    def __init__(self, runnable: Runnable, get_session_history, input_messages_key: str, history_messages_key: str, context_key: Optional[str] = None):
        self.runnable = runnable
        self.get_session_history = get_session_history
        self.input_messages_key = input_messages_key
        self.history_messages_key = history_messages_key
        self.context_key = context_key
    
    def invoke(self, input: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Any:
        session_id = config["configurable"]["session_id"]
        history = self.get_session_history(session_id)
        
        current_input = input[self.input_messages_key]
        
        if isinstance(current_input, str):
            current_input_message = HumanMessage(content=current_input)
            # history.add_messages([current_input_message])
        
        input[self.history_messages_key] = history.messages
        
        result = self.runnable.invoke(input, config)
        
        if isinstance(result, AIMessage):
            if self.context_key and self.context_key in input:
                context = input[self.context_key]
                result_with_context = AIMessage(content=f"{context}\n{result.content}")
                history.add_messages([result_with_context])
            else:
                history.add_messages([result])
        
        return result

In [78]:
def add_memory(runnable, session_id, context="", save_mode="both"):
    runnable_with_memory = RunnableWithMessageHistory(
        runnable,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        context_key="context"
    )
    
    memory_by_session = RunnableLambda(
        lambda input: runnable_with_memory.invoke(
            {**input, "context": context},
            config={"configurable": {"session_id": session_id,
                                     "save_mode": save_mode}}
        )
    )
    return memory_by_session

In [79]:
SESSION_ID = "working-test"

In [80]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into one of the following two types:
            - Product inquiry, order history inquiry, order change history inquiry, order cancellation history inquiry: '문의'
            - Order request, order change request, order cancellation request: '요청'
            
            You need to review the messages in the Messages Placeholder from the latest to the oldest.

            Consider the previous AI responses and their classifications to understand the intent behind the current input. 
            Use this context to make an accurate classification. 
            If the latest AI response was classified as '요청', and the current input is related to an order, it is likely a '요청'.
            
            Additionally, if the input contains order details, it should be classified as '요청'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [81]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [82]:
classify_message_chain = message_type_prompt | model

classify_message_with_memory = add_memory(classify_message_chain, SESSION_ID, context="사용자 입력 유형 분류", save_mode="both")
classify_message_with_memory 

RunnableLambda(lambda input: runnable_with_memory.invoke({**input, 'context': context}, config={'configurable': {'session_id': session_id, 'save_mode': save_mode}}))

In [85]:
classify_message_with_memory.invoke({"input": "물건 반품 가능??"})

AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 205, 'total_tokens': 207}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-12741c21-4f39-449a-9c3e-d4afbba02594-0')

In [86]:
store["working-test"].messages

[AIMessage(content='사용자 입력 유형 분류\n문의'), AIMessage(content='사용자 입력 유형 분류\n문의')]

In [43]:

classify_message_with_memory_chain = RunnablePassthrough.assign(msg_type=classify_message_with_memory)

In [46]:
response = classify_message_with_memory_chain.invoke(
    {"input": "주문 변경하려면 어케 하죠?"}
)
response

{'input': '주문 변경하려면 어케 하죠?',
 'msg_type': AIMessage(content='사용자 입력 유형\n요청', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 301, 'total_tokens': 313}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f409ef6f-2157-4f6e-9ed3-81aaf1e9d47c-0')}

In [47]:
store["working-test"].messages

[HumanMessage(content='주문 취소하려면 어케 하죠?'),
 AIMessage(content='사용자 입력 유형\n요청'),
 HumanMessage(content='주문 취소하려면 어케 하죠?'),
 AIMessage(content='사용자 입력 유형\n사용자 입력 유형\n요청'),
 HumanMessage(content='주문 변경하려면 어케 하죠?'),
 AIMessage(content='사용자 입력 유형\n사용자 입력 유형\n요청')]